In [12]:
import pandas as pd
import openpyxl as op

### Задание 7

In [ ]:
dfe = pd.read_excel('../data/документация/КТ_Кировский.xlsx')
dfc = pd.read_csv('../data/документация/sp.csv')

dfec = dfe.copy(deep=True)
dfcc = dfc.copy(deep=True)

dfcc['Сумма'] = dfcc['Сумма'].replace(',', '.', regex=True).astype(float)
dfcc['Пени'] = dfcc['Пени'].replace(',', '.', regex=True).astype(float)
dfcc['ГосПошлина'] = dfcc['ГосПошлина'].replace(',', '.', regex=True).astype(float)

In [ ]:
df = dfcc.copy(deep=True)
df['Номер ИД'] = df['Номер ИД'].str.split(r',\s*')
df = df.explode('Номер ИД')

df[df['Номер ИД'] == "2-88-1941/2022"], df[df['Номер ИД'] == "2-88-1942/2022"]

In [ ]:
merged_df = pd.merge(dfec, dfcc[['Номер ИД', 'Сумма', 'Пени', 'ГосПошлина']], how='left').drop_duplicates()
merged_df['Общий долг'] = merged_df[['Сумма', 'Пени', 'ГосПошлина']].sum(axis=1)

print(f"Null - {merged_df[merged_df['Общий долг'].isna()].shape}, All - {merged_df.shape}")

Null - (0, 35), All - (2520, 35)


In [ ]:
pd.set_option('display.max_columns', None)
merged_df[merged_df['Общий долг'] == 0]

In [108]:
merged_df.to_excel('../data/документация/КТ_Кировский_ОбщийДолг.xlsx')

In [72]:
dfcc.dtypes

Номер ИД       object
Номер           int64
Счет          float64
Сумма         float64
Пени          float64
ГосПошлина    float64
dtype: object

### Задание 8 - Общий отчет

In [ ]:
fz = pd.read_csv("../data/документация/fz.csv")
sv = pd.read_excel("../data/документация/Стадии взысканий.xlsx")

fzcopy = fz.copy(deep=True)
fzcopy = pd.merge(fzcopy, sv, how="left")

fzcopy['Сумма'] = pd.to_numeric(fzcopy['Сумма'].replace(',', '.', regex=True).astype(float))
fzcopy['ROW_ID_Взыскания'] = pd.to_numeric(fzcopy['ROW_ID_Взыскания'], errors='coerce').fillna(0).astype(int)
fzcopy['НомерДела'] = pd.to_numeric(fzcopy['НомерДела'], errors='coerce').fillna(0).astype(int)
fzcopy['Месяц долга'] = pd.to_datetime(fzcopy['Месяц долга'])


filter1 = pd.read_excel("../data/документация/Абонент Абонент Абонент.xlsx")
filter2 = pd.read_excel("../data/документация/Лица Призванные Наследовать.xlsx")
filter3 = pd.read_excel("../data/документация/Наниматель Жилого Помещения.xlsx")
filter4 = pd.read_excel("../data/документация/Юридические Лица.xlsx")
filter5 = pd.read_excel("../data/документация/Нежилое Помещение.xlsx")
total_filter = pd.concat([filter1, filter2, filter3, filter4, filter5])

df_ost = pd.read_csv("../data/документация/ost1.csv")
df_ces = pd.read_csv("../data/документация/ces1.csv")
df_mun = pd.read_csv("../data/документация/mun1.csv")

def CategoryDZ(df):
    
    date = pd.to_datetime('2025-02-01')

    df['months_diff'] = (
        (date.year - df['Месяц долга'].dt.year) * 12
        + (date.month - df['Месяц долга'].dt.month)
    )

    df['years_diff'] = df['months_diff'] // 12

    def create_row(aLS, aNS, sum):
        return pd.DataFrame([{'КоличествоЛЦ': aLS, 'КоличествоДел': aNS, 'СуммаДолга': sum}])

    dfc = pd.DataFrame(columns=['КоличествоЛЦ', 'КоличествоДел', 'СуммаДолга'])

    # Отчетный месяц
    dfc = pd.concat([dfc, create_row(
                                    df.query('months_diff == 0')['Номер ЛЦ'].nunique(),
                                    df.query('months_diff == 0')['ROW_ID_Взыскания'].nunique(),
                                    df.query('months_diff == 0')['Сумма'].sum())])

    # от 2 до 3 месяцев после отчетного
    dfc = pd.concat([dfc, create_row(
                                    df.query('3 > months_diff >= 2')['Номер ЛЦ'].nunique(),
                                    df.query('3 > months_diff >= 2')['ROW_ID_Взыскания'].nunique(),
                                    df.query('3 > months_diff >= 2')['Сумма'].sum())])

    # от 3 месяцев до 3 лет после отчетного
    dfc = pd.concat([dfc, create_row(
                                    df.query('months_diff >= 3 and years_diff <= 3')['Номер ЛЦ'].nunique(),
                                    df.query('months_diff >= 3 and years_diff <= 3')['ROW_ID_Взыскания'].nunique(),
                                    df.query('months_diff >= 3 and years_diff <= 3')['Сумма'].sum())])

    # свыше 3 лет
    dfc = pd.concat([dfc, create_row(
                                    df.query('years_diff >= 3')['Номер ЛЦ'].nunique(),
                                    df.query('years_diff >= 3')['ROW_ID_Взыскания'].nunique(),
                                    df.query('years_diff >= 3')['Сумма'].sum())])

    # не было оплат
    dfc = pd.concat([dfc, create_row(
                                    0,
                                    0,
                                    0)])
    
    # Свыше 3-х лет+все остальные периоды по этим (закрытым) л/с
    dfc = pd.concat([dfc, create_row(
                                    0,
                                    0,
                                    0)])
    
    # ДЗ менее 3000 р 
    small_sum = df.groupby(['Номер ЛЦ'])['Сумма'].sum().reset_index() #доделать
    small_sum = small_sum[small_sum['Сумма'] < 3000]
    
    small_sum_filter = df[df['Номер ЛЦ'].isin(small_sum['Номер ЛЦ'])]
    
    dfc = pd.concat([dfc, create_row(
                                    len(small_sum),  
                                    small_sum_filter['ROW_ID_Взыскания'].nunique(),            
                                    small_sum['Сумма'].sum())]) 
    
    # Без ФИО, Лица призванные наследовать, Юридические лица, Нежилое помещение, Наниматель жилого помещения, Несовершеннолетние
    dfcopy_filtered = df[df["ФИО"].isin(total_filter["ФИО"])]
    dfc = pd.concat([dfc, create_row(
                                    dfcopy_filtered['Номер ЛЦ'].nunique(),
                                    dfcopy_filtered['ROW_ID_Взыскания'].nunique(),
                                    dfcopy_filtered['Сумма'].sum())])
    return dfc


def Category(df, start_row):
    dfcopy = df.copy(deep=True)
    dfcopy = pd.merge(dfcopy, sv, how="left")

    def updateExcel(df, start_row):

        wba = wb.active
        start_col = 6 

        for row_index, (index, row) in enumerate(df.iterrows(), start=start_row):  
            for col_index, (col_name, cell_value) in enumerate(row.items(), start=start_col):
                cell = wba.cell(row=row_index, column=col_index)
                cell.value = cell_value
        

    dfcopy['Сумма'] = pd.to_numeric(dfcopy['Сумма'].replace(',', '.', regex=True).astype(float))
    dfcopy['ROW_ID_Взыскания'] = pd.to_numeric(dfcopy['ROW_ID_Взыскания'], errors='coerce').fillna(0).astype(int)
    dfcopy['НомерДела'] = pd.to_numeric(dfcopy['НомерДела'], errors='coerce').fillna(0).astype(int)
    dfcopy['Месяц долга'] = pd.to_datetime(dfcopy['Месяц долга'])

    # Досудебная
    Dosud = dfcopy[(dfcopy["Общая стадия"] == "Досубедная от месяца") | dfcopy["Фаза"].isna() | dfcopy["Общая стадия"].isna()]

    # Судебная
    Sud = dfcopy[dfcopy["Общая стадия"] == "Суд"]

    # ФССП
    Fssp = dfcopy[(dfcopy["Общая стадия"] == "ФССП")]

    total_df = pd.concat([CategoryDZ(Dosud), CategoryDZ(Sud), CategoryDZ(Fssp)], axis=1)

    updateExcel(total_df, start_row)
    return total_df


wb = op.load_workbook('../data/документация/Отчет ежемесячный КT2(копия).xlsx')
Category(df_ost, 17)
Category(df_ces, 27)
Category(df_mun, 37)
wb.save('../data/документация/test8.xlsx')

C:\Users\n.kostryukov\AppData\Local\Temp\ipykernel_74172\2868314163.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['months_diff'] = (
C:\Users\n.kostryukov\AppData\Local\Temp\ipykernel_74172\2868314163.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['years_diff'] = df['months_diff'] // 12
C:\Users\n.kostryukov\AppData\Local\Temp\ipykernel_74172\2868314163.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will 

TypeError: Workbook.save() missing 1 required positional argument: 'filename'

In [89]:
df_ces

,Сумма,Номер ЛЦ,Счет-Наниматель,Месяц долга,Наименование,ROW_ID_Организации,ФИО,Дата рождения,Дата смерти,ROW_ID_Взыскания,Фаза,Дата постановки на фазу,ДатНачДолга,ДатКнцДолга,НомерДела
0,"3048,52",1426,672400,2017-01-01 00:00:00,"Цессии ООО ""Концессии теплоснабжения"" (дог. 14...",4755,Бавыкина Надежда Ефимовна,1948-10-19 00:00:00.000,NaN,205477.0,Исполнительное производство Сдача в ФССП,2021-06-19 00:00:00.000,2016-10-01 00:00:00.000,2017-03-31 00:00:00.000,2.000000e+00
1,"348,58",1426,672400,2017-07-01 00:00:00,"Цессии ООО ""Концессии теплоснабжения"" (дог. 14...",4755,Бавыкина Надежда Ефимовна,1948-10-19 00:00:00.000,NaN,258285.0,Исполнительное производство Сдача в ФССП,2021-06-19 00:00:00.000,2017-04-01 00:00:00.000,2018-03-01 00:00:00.000,2.147761e+09
2,"1151,75",1129,671966,2016-10-01 00:00:00,"Цессии ООО ""Концессии теплоснабжения"" (дог. 14...",4755,Селезнева Валентина Александровна,1980-04-23 00:00:00.000,NaN,205427.0,Исполнительное производство Сдача в ФССП,2021-06-19 00:00:00.000,2016-10-01 00:00:00.000,2017-03-31 00:00:00.000,2.000000e+00
3,"1362,34",1129,671966,2017-04-01 00:00:00,"Цессии ООО ""Концессии теплоснабжения"" (дог. 14...",4755,Селезнева Валентина Александровна,1980-04-23 00:00:00.000,NaN,502144.0,Исполнительное производство Сдача в ФССП,2017-12-14 00:00:00.000,2017-04-01 00:00:00.000,2017-06-01 00:00:00.000,2.147754e+09
4,"697,16",1129,671966,2017-07-01 00:00:00,"Цессии ООО ""Концессии теплоснабжения"" (дог. 14...",4755,Селезнева Валентина Александровна,1980-04-23 00:00:00.000,NaN,497830.0,Исполнительное производство Сдача в ФССП,2018-10-25 00:00:00.000,2017-07-01 00:00:00.000,2017-12-01 00:00:00.000,2.147750e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167264,"151,34",2008095107,1203163,2017-06-01 00:00:00,"Цессии ООО ""Концессии теплоснабжения"" (дог. 14...",4755,Плетнева Диана Владимировна,1989-05-29 00:00:00.000,NaN,223531.0,Исполнительное производство Сдача в ФССП,2021-06-19 00:00:00.000,2016-10-01 00:00:00.000,2017-06-30 00:00:00.000,2.000000e+00
167265,"134,97",2008098207,1203229,2017-06-01 00:00:00,"Цессии ООО ""Концессии теплоснабжения"" (дог. 14...",4755,Жаринова Светлана Вадимовна,1989-07-26 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167266,"1298,08",2008099607,1203257,2017-01-01 00:00:00,"Цессии ООО ""Концессии теплоснабжения"" (дог. 14...",4755,Дудкина Анна Владимировна,1977-04-20 00:00:00.000,NaN,235793.0,Исполнительное производство Сдача в ФССП,2021-06-19 00:00:00.000,2016-10-01 00:00:00.000,2017-06-30 00:00:00.000,2.000000e+00
167267,"591,8",2008099607,1203257,2017-04-01 00:00:00,"Цессии ООО ""Концессии теплоснабжения"" (дог. 14...",4755,Дудкина Анна Владимировна,1977-04-20 00:00:00.000,NaN,235793.0,Исполнительное производство Сдача в ФССП,2021-06-19 00:00:00.000,2016-10-01 00:00:00.000,2017-06-30 00:00:00.000,2.000000e+00


In [ ]:
Category(df_ost, 17)

In [ ]:
Category(df_ces, 27)

In [ ]:
Category(df_mun, 37)

In [80]:
df_ost.shape[0] + df_ces.shape[0] + df_mun.shape[0]

987369

In [81]:
fzcopy[fzcopy["ФИО"].isin(total_filter["ФИО"])].shape[0]

71982

In [5]:
fzcopy.dtypes

Сумма                             float64
Номер ЛЦ                            int64
Счет-Наниматель                   float64
Месяц долга                datetime64[ns]
Наименование                       object
ROW_ID_Организации                  int64
ФИО                                object
Дата рождения                      object
Дата смерти                        object
ROW_ID_Взыскания                    int64
Фаза                               object
Дата постановки на фазу            object
ДатНачДолга                        object
ДатКнцДолга                        object
НомерДела                           int64
Общая стадия                       object
Сомнительность                     object
dtype: object